In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

data = pd.read_csv("/content/gdrive/MyDrive/Churn_Modelling.csvChurn_Modelling.csv",delimiter=',')

In [6]:
X = data.iloc[:,3:-1].values
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [8]:
#Generating Dependent Variable Vectors
Y = data.iloc[:,-1].values

print(Y)

[1 0 1 ... 1 1 0]


In [12]:
#Encoding Categorical Variable Gender
from sklearn.preprocessing import LabelEncoder
LE1 = LabelEncoder()
X[:,2] = np.array(LE1.fit_transform(X[:,2]))
print(X)


[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [13]:
#One-hot encoding ensures that the machine learning algorithm does not assume that higher numbers are more important.
#Encoding Categorical variable Geography
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct =ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder="passthrough")
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [14]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [15]:
#Performing Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
#Initialising ANN
ann = tf.keras.models.Sequential()
#Adding First Hidden Layer
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))
#Adding Second Hidden Layer
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))
#Adding Output Layer
ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))
#Compiling ANN
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [17]:
#convert all values to float32 format
X_train = np.asarray(X_train).astype(np.float32)
Y_train = np.asarray(Y_train).astype(np.float32)
#Fitting ANN
ann.fit(X_train,Y_train,batch_size=32,epochs = 100)

Epoch 1/100
250/250 [==============================] - 6s 3ms/step - loss: 0.5580 - accuracy: 0.7600
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4914 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4682 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4541 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4437 - accuracy: 0.7961
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4366 - accuracy: 0.7980
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4304 - accuracy: 0.8025
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4243 - accuracy: 0.8092
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4165 - accuracy: 0.8167
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4047 - accura

In [18]:
#Predicting result for Single Observation
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1,50000]])) > 0.5)
#convert all values to float32 format
X = np.asarray(X).astype(np.float32)
Y = np.asarray(Y).astype(np.float32)

1/1 [==============================] - 0s 106ms/step
[[False]]


In [19]:
# make class predictions with the model
predictions = (ann.predict(X) > 0.5).astype(int)
# summarize the first 5 cases
for i in range(5):
 print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], Y[i]))

313/313 [==============================] - 1s 2ms/step
[1.0, 0.0, 0.0, 619.0, 0.0, 42.0, 2.0, 0.0, 1.0, 1.0, 1.0, 101348.8828125] => 0 (expected 1)
[0.0, 0.0, 1.0, 608.0, 0.0, 41.0, 1.0, 83807.859375, 1.0, 0.0, 1.0, 112542.578125] => 1 (expected 0)
[1.0, 0.0, 0.0, 502.0, 0.0, 42.0, 8.0, 159660.796875, 3.0, 1.0, 0.0, 113931.5703125] => 1 (expected 1)
[1.0, 0.0, 0.0, 699.0, 0.0, 39.0, 1.0, 0.0, 2.0, 0.0, 0.0, 93826.6328125] => 0 (expected 0)
[0.0, 0.0, 1.0, 850.0, 0.0, 43.0, 2.0, 125510.8203125, 1.0, 1.0, 1.0, 79084.1015625] => 1 (expected 0)


In [20]:
# evaluate the keras model
_, accuracy = ann.evaluate(X, Y)
print('Accuracy: %.2f' % (accuracy*100))

313/313 [==============================] - 1s 2ms/step - loss: 15899.1084 - accuracy: 0.3937
Accuracy: 39.37
